In [2]:
#require "nomad";;

open Nomad.Expr
open Nomad.Simplify
open Nomad.Eval
open Nomad.Derivatives
open Nomad.Gradient_descent
open Nomad.Newton
open Nomad.Bisection

In [5]:
(* Zadanie z AN - lista 13 zad. 3 *)
let e = Cos ((Float 3. *: Var "x") +: (Var "pi" /: Float 4.)) in
let e' = nth_derivative e "x" 4 in
string_of_expr e';;

- : string = "81. * cos(0.25 * pi + 3. * x)"


In [7]:
let e = (Float 1. *: Var "y" *: Var "y") +: (Float 2. *: Var "y" *: Float 2.);;

simplify e |> string_of_expr;;

val e : Nomad.Expr.expr =
  Sum
   [Product [Product [Float 1.; Var "y"]; Var "y"];
    Product [Product [Float 2.; Var "y"]; Float 2.]]


- : string = "4. * y + y^2."


In [9]:
let e = Float 2. *: Var "y" *: Float 2.;;

e |> collect |> string_of_expr;;

val e : Nomad.Expr.expr = Product [Product [Float 2.; Var "y"]; Float 2.]


- : string = "4. * y"


In [10]:
let e = Product ([Float 1.; Var "y"; Float 2.; Var "x"; Float 3.;  Var "x"; Var "y"]) in 

simplify e |> string_of_expr;;

- : string = "6. * x^2. * y^2."


In [12]:
let e = Float 3. *: Var "z" /: Var "z";;

simplify e |> string_of_expr;;

val e : Nomad.Expr.expr =
  Product [Product [Float 3.; Var "z"]; Pow (Var "z", Float (-1.))]


- : string = "3."


In [4]:
(* Financial Mathematics - Black-Scholes Option Pricing *)
let black_scholes_example () =
  let s = Var "S" in        (* Stock price *)
  let k = Float 100.0 in    (* Strike price *)
  let r = Float 0.05 in     (* Risk-free rate *)
  let t = Var "t" in        (* Time to expiration *)
  let sigma = Float 0.2 in  (* Volatility *)
  
  (* Call option price formula (simplified) *)
  let d1 = (Log(s /: k) +: (r +: (sigma ^: Float 2.) /: Float 2.) *: t) /: 
           (sigma *: Pow(t, Float 0.5)) in
  
  let derivative = derivative d1 "S" |> simplify in
  
  print_endline "Black-Scholes Option Delta:";
  print_endline (string_of_expr derivative);
  print_endline "Value at S=100, t=1:";
  print_endline (string_of_float (eval [("S", 100.0); ("t", 1.0)] derivative))

  
let () = black_scholes_example ()

val black_scholes_example : unit -> unit = <fun>


Black-Scholes Option Delta:
0.01 / 0.01 * S / 0.2 * t^0.5 + -1. / 0.2 * t^0.5 * 0.2 * 0.5 / t * (0.07 * t + log(0.01 * S))
Value at S=100, t=1:
-0.125


In [5]:
(* Machine Learning - Neural Network Analysis *)
let neural_network_example () =
  (* ReLU activation function and its derivative *)
  let relu x = Max(x, Float 0.) in
  let x = Var "x" in
  let hidden_layer = [relu (x *: Float 2.); relu (x *: Float (-1.) +: Float 1.)] in
  let output = List.fold_left (+:) (Float 0.) hidden_layer in
  
  let gradient = derivative output "x" |> simplify in
  
  print_endline "Neural Network with ReLU Activation:";
  print_endline ("f(x) = " ^ latex_of_expr output);
  print_endline ("f'(x) = " ^ latex_of_expr gradient)

error: compile_error

In [36]:
(* Physics - Projectile Motion with Air Resistance *)
let projectile_motion_example () =
  let t = Var "t" in
  let v0 = Float 50.0 in  (* Initial velocity *)
  let theta = Float (3.14 /. 4.) in  (* Launch angle *)
  let g = Float 9.81 in
  let k = Float 0.1 in  (* Air resistance coefficient *)
  
  (* Position equations *)
  let x = v0 *: Cos(theta) *: t -: (k /: Float 2.) *: Pow(t, Float 2.) in
  let y = v0 *: Sin(theta) *: t -: (g /: Float 2.) *: Pow(t, Float 2.) in
  
  print_endline "Projectile Motion Equations:";
  print_endline ("x(t) = " ^ latex_of_expr x);
  print_endline ("y(t) = " ^ latex_of_expr y);
  
  (* Find time of landing *)
  let eq = (y, Float 0.) in
  match solve_newton eq ~initial_guess:5.0 ~max_iter:100 with
  | Ok t_land -> print_endline ("Landing time: " ^ string_of_float t_land)
  | Error msg -> print_endline msg

error: compile_error

In [37]:
(* Engineering - Beam Deflection Analysis *)
let beam_deflection_example () =
  let x = Var "x" in
  let L = Float 10.0 in  (* Beam length *)
  let EI = Float 2000.0 in  (* Flexural rigidity *)
  let w = Float 1.0 in  (* Distributed load *)
  
  (* Beam deflection equation *)
  let deflection = (w *: Pow(x, Float 4.)) /: (Float 24. *: EI) -:
                   (w *: L *: Pow(x, Float 3.)) /: (Float 12. *: EI) in
  
  let max_deflection = derivative deflection "x" |> simplify in
  
  print_endline "Beam Deflection Analysis:";
  print_endline ("y(x) = " ^ latex_of_expr deflection);
  print_endline "Maximum deflection occurs at:";
  
  (* Find point of maximum deflection *)
  let eq = (max_deflection, Float 0.) in
  match solve_newton eq ~initial_guess:5.0 ~max_iter:100 with
  | Ok x_max -> print_endline ("x = " ^ string_of_float x_max)
  | Error msg -> print_endline msg

error: compile_error

In [38]:
(* Run examples *)
let () =
  print_endline "\n=== Financial Mathematics ===\n";
  black_scholes_example ();
  
  print_endline "\n=== Machine Learning ===\n";
  neural_network_example ();
  
  print_endline "\n=== Physics ===\n";
  projectile_motion_example ();
  
  print_endline "\n=== Engineering ===\n";
  beam_deflection_example ()

error: compile_error

In [27]:
(* Example 1: Neural Network Activation Function Analysis *)
let sigmoid x = Div(Float 1., Float 1. +: Exp(Float (-1.) *: x)) in
let x = Var "x" in
let expr = sigmoid x in
let derivative_expr = derivative expr "x" |> simplify in

make_output "Neural Network Activation Function" (
  "Original sigmoid: " ^ string_of_expr expr ^ "\n" ^
  "Derivative: " ^ string_of_expr derivative_expr ^ "\n" ^
  "LaTeX: " ^ latex_of_expr derivative_expr
) |> print_endline;;



Neural Network Activation Function
Original sigmoid: 1. / (1. + exp(-1. * x))
Derivative: (0. - -1. * exp(-1. * x)) / ((exp(-1. * x) + 1.) * (exp(-1. * x) + 1.))
LaTeX: \frac{0. - -1. \cdot e^{-1. \cdot x}}{(e^{-1. \cdot x} + 1.) \cdot (e^{-1. \cdot x} + 1.)}



- : unit = ()
